# **Mount Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

# **Load Model**

In [ ]:
!pip install chardet --quiet

In [ ]:
import transformers
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig, MBartForConditionalGeneration, MBart50Tokenizer, MBartTokenizer
import torch
import pandas as pd
import chardet

In [ ]:
model_name = "facebook/mbart-large-50"

# Load the tokenizer and model from Hugging Face
tokenizer = MBart50Tokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

# **Create Prompt**

In [ ]:
# System Prompt with Example
system_prompt = """
You are a Thai language expert in finance stock market model. Your task is to indentify symbols and extract aspect, opinion word, and sentiment from a given sentence that is an opinion about The Stock Exchange of Thailand.
Task:
For each sentence, you need to identify:

The symbol, it is abbreviation name for a stock on the stock market.
The aspect term related to the stock market (if explicitly present or inferred from context).
The opinion words related to the aspect (if explicitly present or inferred from context).
The sentiment associated with the aspect and opinion (positive or negative or neutral).

Special Instructions:
If symbol is not explicitly mentioned, skip this sentence and don't include it in the output.
If aspect and opinion word is not explicitly mentioned, provide the most probable one based on the context about stock market.
If a symbol in sentence has multiple aspects or opinions, list each aspect separately in the output.
Format the output exactly as shown in the examples outputs.

The output should be a list of lists, where each inner list contains:
The <symbol>, <aspect>, <opinion>, and <sentiment> tag for each list.
The symbol which is abbreviation name for a stock on the stock market.
The aspect term which is related to stock market.
The opinion words related to the aspect.
The sentiment ('POS' for positive, 'NEG' for negative, 'NEU' for neutral).

Example Outputs:
Input: "บล.ดาโอแนะนำ "ซื้อ" ITC ต้นทุนทูน่าลดต่อเนื่อง"
Output: [[<symbol>ITC<aspect>แนะนำ<opinion>ซื้อ<sentiment>POS],[<symbol>ITC<aspect>ต้นทุน<opinion>ลดต่อเนื่อง<sentiment>POS]]

Input: "PACO ลั่นยอดโต20% รับอานิสงส์จีนย้ายฐาน"
Output: [<symbol>PACO<aspect>ยอด<opinion>โต<sentiment>POS]

Input: "PR9 แนวโน้มรายได้-อัตรากำไรดีขึ้น บล.กสิกรฯเพิ่มคำแนะนำเป็น “ซื้อ”"
Output: [[<symbol>PACO<aspect>รายได้<opinion>โต<sentiment>POS],[<symbol>PACO<aspect>อัตรากำไร<opinion>ดีขึ้น<sentiment>POS],[<symbol>PACO<aspect>คำแนะนำ<opinion>ซื้อ<sentiment>POS]]
"""

# **Export to csv**

In [ ]:
def generate_response(model, tokenizer, prompt, max_new_tokens=70, temperature=1.2, top_p=0.95):

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]

    input_ids = tokenizer.apply_chat_template(messages,add_generation_prompt=True,padding=True,truncation=True,return_tensors="pt").to(model.device)


    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        pad_token_id=model.config.eos_token_id,
        eos_token_id=terminators,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
    )

    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

In [ ]:
# Read a small part of the file to detect the encoding
with open('/content/drive/MyDrive/ABSA_Datasets/Text-to-Label-Dataset.csv', 'rb') as f:
    result = chardet.detect(f.read(10000))

# Print the detected encoding
print(result['encoding'])

# Read the CSV file using the detected encoding
df = pd.read_csv('/content/drive/MyDrive/ABSA_Datasets/Text-to-Label-Dataset.csv', encoding=result['encoding'])

utf-8


In [ ]:
def process_typhoon(sentence):
    #prompt = f"**Input:** \"{sentence}\""
    result = generate_response(model,tokenizer,sentence)
    return result

# Apply the function to the 'sentence' column and store results in a new column
df['mBART_adjusted_prompt'] = df['text'].apply(process_typhoon)

df.to_csv('/content/drive/MyDrive/ABSA_Datasets/Mbart_prompt.csv', index=False, encoding='utf-8-sig')

ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating